# KAFKA PRODUCER

In [4]:
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
import tweepy
import os
import json
from kafka import KafkaProducer

ACCESS_TOKEN=os.getenv('ACCESS_TOKEN')
ACCESS_TOKEN_SECRET=os.getenv('ACCESS_TOKEN_SECRET')
CONSUMER_KEY=os.getenv('CONSUMER_KEY')
CONSUMER_SECRET=os.getenv('CONSUMER_SECRET')

#SEARCH_KEYWORD = str(input("Please enter a keyword to filter\n"))

producer = KafkaProducer(bootstrap_servers=['kafka:9092'])

def filter_tweet(tweet):
    data = json.loads(tweet)
    
    if 'delete' in data:
        return False
    if 'timestamp_ms' not in data or 'entities' not in data:
        return False      
    if len(data['entities']['hashtags']) <= 0:
        return False
        
    data = {"timestamp": data['timestamp_ms'], "hashtags": [i["text"] for i in data['entities']['hashtags']]}
    print(data)
    data = json.dumps(data)
    
    return data


class StdOutListener(StreamListener):
    def __init__(self):
        self.count=0

    def on_data(self, data):
        data = filter_tweet(data)
        if data != False:
            producer.send("tweets", data.encode('utf-8'))  
         
        self.count+=1 
        print(self.count)
        return True
    def on_error(self, status):
        print(status)
        
    def on_exception(self, exception):
        print(exception)
        return

myStreamListener = StdOutListener()
auth = OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
myStream = tweepy.Stream(auth = auth, listener=myStreamListener)
#myStream.filter(track=[SEARCH_KEYWORD])
myStream.filter(languages=["en"], track=["a", "the", "i", "you", "u"])
#myStream.sample(languages=["en"])

print(f"Starting tweet streams")

1
2
3
4
5
6
7
8
9
{'timestamp': '1624626779616', 'hashtags': ['원호', 'WONHO']}
10
11
12
13
14
15
16
17
18
19
20
21
22
{'timestamp': '1624626779589', 'hashtags': ['Solana', 'Solanaairdrop', 'Airdrop']}
23
24
25
26
27
28
29
{'timestamp': '1624626779628', 'hashtags': ['Bitcoin']}
30
31
32
33
34
35
{'timestamp': '1624626779592', 'hashtags': ['SaveAleppo']}
36
{'timestamp': '1624626779648', 'hashtags': ['VERNON']}
37
38
39
{'timestamp': '1624626779645', 'hashtags': ['KuttiStory', 'Master']}
40
41
42
43
44
45
46
47
48
{'timestamp': '1624626779658', 'hashtags': ['ITSALLINTHEMIX', 'giveaway']}
49
{'timestamp': '1624626779627', 'hashtags': ['Solana', 'Solanaairdrop', 'Airdrop']}
50
51
{'timestamp': '1624626779593', 'hashtags': ['뮤직뱅크', 'ITZY', '있지', 'MIDZY', '믿지', 'GUESSWHO']}
52
53
54
55
56
{'timestamp': '1624626779620', 'hashtags': ['EUMercosur']}
57
58
59
60
61
62
63
64
{'timestamp': '1624626779677', 'hashtags': ['SAFETREES', 'ClimateAction', 'savetheplanet']}
65
66
67
68
69
70
71
72
{'timest

KeyboardInterrupt: 